<div style="display: flex; background-color: #3F579F;">
    <h1 style="margin: auto; font-weight: bold; padding: 30px 30px 0px 30px; color:#fff;" align="center">Implement a scoring model - P7</h1>
</div>
<div style="display: flex; background-color: #3F579F; margin: auto; padding: 5px 30px 0px 30px;" >
    <h3 style="width: 100%; text-align: center; float: left; font-size: 24px; color:#fff;" align="center">| Preprocessing Notebook |</h3>
</div>
<div style="display: flex; background-color: #3F579F; margin: auto; padding: 10px 30px 30px 30px;">
    <h4 style="width: 100%; text-align: center; float: left; font-size: 24px; color:#fff;" align="center">Data Scientist course - OpenClassrooms</h4>
</div>

<div class="alert alert-block alert-info">
    <p>Preprocessing (Feature Engineering) based on the Kernel made for the Home Credit's
competition in Kaggle.  <a href="https://www.kaggle.com/jsaguiar/lightgbm-with-simple-features/script", target="blank">[LightGBM with Simple Features]</a></p>
    <p>Based on this Feature Engineering, we are going to try to optimize the memory usage and deal with missing-values.</p>
</div>

<div style="background-color: #506AB9;" >
    <h2 style="margin: auto; padding: 20px; color:#fff; ">1. Libraries</h2>
</div>

<div style="background-color: #506AB9;" >
    <h3 style="margin: auto; padding: 20px; color:#fff; ">1.1 Libraries</h3>
</div>

In [1]:
import gc
import time
import re
import warnings
from contextlib import contextmanager
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

warnings.simplefilter(action='ignore', category=FutureWarning)

## Own specific functions 
from functions import *

<div style="background-color: #506AB9;" >
    <h2 style="margin: auto; padding: 20px; color:#fff; ">2. Functions</h2>
</div>

<div style="background-color: #6D83C5;" >
    <h3 style="margin: auto; padding: 20px; color:#fff; ">Timer</h3>
</div>

In [2]:
@contextmanager
def timer(title):
    t0 = time.time()
    yield
    print("{} - done in {:.0f}s".format(title, time.time() - t0))

<div style="background-color: #6D83C5;" >
    <h3 style="margin: auto; padding: 20px; color:#fff; ">Categorical encoder</h3>
</div>

<div class="alert alert-block alert-info">
    <p>One-hot encoding for categorical columns with get_dummies</p>
    <p>We are going to add "COL_CAT_" as prefix for each categorical column added</p>
</div>

In [3]:
# One-hot encoding for categorical columns with get_dummies
def one_hot_encoder(df, nan_as_category=True):
    original_columns = list(df.columns)
    categorical_columns = [col for col in df.columns if df[col].dtype == 'object']
    df = pd.get_dummies(df, columns=categorical_columns, dummy_na=nan_as_category)
    new_columns = [c for c in df.columns if c not in original_columns]
    return df, new_columns

<div style="background-color: #6D83C5;" >
    <h3 style="margin: auto; padding: 20px; color:#fff; ">Preprocess main table</h3>
</div>

<div class="alert alert-block alert-info">
    <p>Read and merge the train and test datasets</p>
    <p>Transforms boolean and categorical features</p>
    <p>Add new features from existing features</p>
</div>

In [4]:
# Preprocess application_train.csv and application_test.csv
def application_train_test(num_rows=None, nan_as_category=False):
    
    # Read data and merge
    df = pd.read_csv(r"datasets\initials_datasets\application_train.csv", nrows=num_rows)
    test_df = pd.read_csv(r"datasets\initials_datasets\application_test.csv", nrows=num_rows)
    print("Train samples: {}, test samples: {}".format(len(df), len(test_df)))
    df = df.append(test_df).reset_index()
    
    # Optional: Remove 4 applications with XNA CODE_GENDER (train set)
    df = df[df['CODE_GENDER'] != 'XNA']
    
    # Categorical features with Binary encode (0 or 1; two categories)
    for bin_feature in ['CODE_GENDER', 'FLAG_OWN_CAR', 'FLAG_OWN_REALTY']:
        df[bin_feature], uniques = pd.factorize(df[bin_feature])
        
    # Categorical features with One-Hot encode
    df, cat_cols = one_hot_encoder(df, nan_as_category)
    
    # NaN values for DAYS_EMPLOYED: 365.243 -> nan
    df['DAYS_EMPLOYED'].replace(365243, np.nan, inplace=True)
    
    # Some simple new features (percentages)
    df['DAYS_EMPLOYED_PERC'] = df['DAYS_EMPLOYED'] / df['DAYS_BIRTH']
    df['INCOME_CREDIT_PERC'] = df['AMT_INCOME_TOTAL'] / df['AMT_CREDIT']
    df['INCOME_PER_PERSON'] = df['AMT_INCOME_TOTAL'] / df['CNT_FAM_MEMBERS']
    df['ANNUITY_INCOME_PERC'] = df['AMT_ANNUITY'] / df['AMT_INCOME_TOTAL']
    df['PAYMENT_RATE'] = df['AMT_ANNUITY'] / df['AMT_CREDIT']
    
    del test_df
    gc.collect()
    
    return df

<div style="background-color: #6D83C5;" >
    <h3 style="margin: auto; padding: 20px; color:#fff; ">Bureau and Bureau balance</h3>
</div>

<div class="alert alert-block alert-info">
    <p>Previous credit of applicants information in <b>others</b> finance institutions</p>
    <p>Add new features based on Max, Min, Mean based on monthly values of previos loans</p>
</div>

In [5]:
# Preprocess bureau.csv and bureau_balance.csv
def bureau_and_balance(num_rows=None, nan_as_category=True):
    
    # Read data
    bureau = pd.read_csv(r"datasets\initials_datasets\bureau.csv", nrows=num_rows)
    bb = pd.read_csv(r"datasets\initials_datasets\bureau_balance.csv", nrows=num_rows)
    
    # Categorical features with One-Hot encode
    bb, bb_cat = one_hot_encoder(bb, nan_as_category)
    bureau, bureau_cat = one_hot_encoder(bureau, nan_as_category)
    
    # Bureau balance: Perform aggregations and merge with bureau.csv
    bb_aggregations = {'MONTHS_BALANCE': ['min', 'max', 'size']}
    for col in bb_cat:
        bb_aggregations[col] = ['mean']
    bb_agg = bb.groupby('SK_ID_BUREAU').agg(bb_aggregations)
    bb_agg.columns = pd.Index([e[0] + "_" + e[1].upper() for e in bb_agg.columns.tolist()])
    bureau = bureau.join(bb_agg, how='left', on='SK_ID_BUREAU')
    bureau.drop(['SK_ID_BUREAU'], axis=1, inplace= True)
    del bb, bb_agg
    gc.collect()
    
    # Bureau and bureau_balance numeric features
    num_aggregations = {
        'DAYS_CREDIT': ['min', 'max', 'mean', 'var'],
        'DAYS_CREDIT_ENDDATE': ['min', 'max', 'mean'],
        'DAYS_CREDIT_UPDATE': ['mean'],
        'CREDIT_DAY_OVERDUE': ['max', 'mean'],
        'AMT_CREDIT_MAX_OVERDUE': ['mean'],
        'AMT_CREDIT_SUM': ['max', 'mean', 'sum'],
        'AMT_CREDIT_SUM_DEBT': ['max', 'mean', 'sum'],
        'AMT_CREDIT_SUM_OVERDUE': ['mean'],
        'AMT_CREDIT_SUM_LIMIT': ['mean', 'sum'],
        'AMT_ANNUITY': ['max', 'mean'],
        'CNT_CREDIT_PROLONG': ['sum'],
        'MONTHS_BALANCE_MIN': ['min'],
        'MONTHS_BALANCE_MAX': ['max'],
        'MONTHS_BALANCE_SIZE': ['mean', 'sum']
    }
    
    # Bureau and bureau_balance categorical features
    cat_aggregations = {}
    for cat in bureau_cat: cat_aggregations[cat] = ['mean']
    for cat in bb_cat: cat_aggregations[cat + "_MEAN"] = ['mean']
    
    bureau_agg = bureau.groupby('SK_ID_CURR').agg({**num_aggregations, **cat_aggregations})
    bureau_agg.columns = pd.Index(['BURO_' + e[0] + "_" + e[1].upper() for e in bureau_agg.columns.tolist()])
    
    # Bureau: Active credits - using only numerical aggregations
    active = bureau[bureau['CREDIT_ACTIVE_Active'] == 1]
    active_agg = active.groupby('SK_ID_CURR').agg(num_aggregations)
    active_agg.columns = pd.Index(['ACTIVE_' + e[0] + "_" + e[1].upper() for e in active_agg.columns.tolist()])
    bureau_agg = bureau_agg.join(active_agg, how='left', on='SK_ID_CURR')
    del active, active_agg
    gc.collect()
    
    # Bureau: Closed credits - using only numerical aggregations
    closed = bureau[bureau['CREDIT_ACTIVE_Closed'] == 1]
    closed_agg = closed.groupby('SK_ID_CURR').agg(num_aggregations)
    closed_agg.columns = pd.Index(['CLOSED_' + e[0] + "_" + e[1].upper() for e in closed_agg.columns.tolist()])
    bureau_agg = bureau_agg.join(closed_agg, how='left', on='SK_ID_CURR')
    
    del closed, closed_agg, bureau
    gc.collect()
    
    return bureau_agg

<div style="background-color: #6D83C5;" >
    <h3 style="margin: auto; padding: 20px; color:#fff; ">Previous applications</h3>
</div>

<div class="alert alert-block alert-info">
    <p>Previous credit of applicants information in <b>"Prêt à dépenser"</b></p>
    <p>Add new features based on Max, Min, Mean based on monthly values of previos loans</p>
</div>

In [6]:
# Preprocess previous_applications.csv
def previous_applications(num_rows=None, nan_as_category=True):
    
    # Read data
    prev = pd.read_csv(r"datasets\initials_datasets\previous_application.csv", nrows=num_rows)
    
    # Categorical features with One-Hot encode
    prev, cat_cols = one_hot_encoder(prev, nan_as_category= True)
    
    # Days 365.243 values -> nan
    prev['DAYS_FIRST_DRAWING'].replace(365243, np.nan, inplace= True)
    prev['DAYS_FIRST_DUE'].replace(365243, np.nan, inplace= True)
    prev['DAYS_LAST_DUE_1ST_VERSION'].replace(365243, np.nan, inplace= True)
    prev['DAYS_LAST_DUE'].replace(365243, np.nan, inplace= True)
    prev['DAYS_TERMINATION'].replace(365243, np.nan, inplace= True)
    
    # Add feature: value ask / value received percentage
    prev['APP_CREDIT_PERC'] = prev['AMT_APPLICATION'] / prev['AMT_CREDIT']
    
    # Previous applications numeric features
    num_aggregations = {
        'AMT_ANNUITY': ['min', 'max', 'mean'],
        'AMT_APPLICATION': ['min', 'max', 'mean'],
        'AMT_CREDIT': ['min', 'max', 'mean'],
        'APP_CREDIT_PERC': ['min', 'max', 'mean', 'var'],
        'AMT_DOWN_PAYMENT': ['min', 'max', 'mean'],
        'AMT_GOODS_PRICE': ['min', 'max', 'mean'],
        'HOUR_APPR_PROCESS_START': ['min', 'max', 'mean'],
        'RATE_DOWN_PAYMENT': ['min', 'max', 'mean'],
        'DAYS_DECISION': ['min', 'max', 'mean'],
        'CNT_PAYMENT': ['mean', 'sum'],
    }
    
    # Previous applications categorical features
    cat_aggregations = {}
    for cat in cat_cols:
        cat_aggregations[cat] = ['mean']
    
    prev_agg = prev.groupby('SK_ID_CURR').agg({**num_aggregations, **cat_aggregations})
    prev_agg.columns = pd.Index(['PREV_' + e[0] + "_" + e[1].upper() for e in prev_agg.columns.tolist()])
    
    # Previous Applications: Approved Applications - only numerical features
    approved = prev[prev['NAME_CONTRACT_STATUS_Approved'] == 1]
    approved_agg = approved.groupby('SK_ID_CURR').agg(num_aggregations)
    approved_agg.columns = pd.Index(['APPROVED_' + e[0] + "_" + e[1].upper() for e in approved_agg.columns.tolist()])
    prev_agg = prev_agg.join(approved_agg, how='left', on='SK_ID_CURR')
    
    # Previous Applications: Refused Applications - only numerical features
    refused = prev[prev['NAME_CONTRACT_STATUS_Refused'] == 1]
    refused_agg = refused.groupby('SK_ID_CURR').agg(num_aggregations)
    refused_agg.columns = pd.Index(['REFUSED_' + e[0] + "_" + e[1].upper() for e in refused_agg.columns.tolist()])
    prev_agg = prev_agg.join(refused_agg, how='left', on='SK_ID_CURR')
    
    del refused, refused_agg, approved, approved_agg, prev
    gc.collect()
    
    return prev_agg

<div style="background-color: #6D83C5;" >
    <h3 style="margin: auto; padding: 20px; color:#fff; ">Pos Cash
</div>

<div class="alert alert-block alert-info">
    <p>Monthly credit (home credit, cash and consumer credits) balances of the customer</p>
</div>

In [7]:
# Preprocess POS_CASH_balance.csv
def pos_cash(num_rows=None, nan_as_category=True):
    
    # Read data
    pos = pd.read_csv(r"datasets\initials_datasets\POS_CASH_balance.csv", nrows=num_rows)
    
    # Categorical features with One-Hot encode
    pos, cat_cols = one_hot_encoder(pos, nan_as_category=True)
    
    # Features
    aggregations = {
        'MONTHS_BALANCE': ['max', 'mean', 'size'],
        'SK_DPD': ['max', 'mean'],
        'SK_DPD_DEF': ['max', 'mean']
    }
    
    for cat in cat_cols:
        aggregations[cat] = ['mean']
    
    pos_agg = pos.groupby('SK_ID_CURR').agg(aggregations)
    pos_agg.columns = pd.Index(['POS_' + e[0] + "_" + e[1].upper() for e in pos_agg.columns.tolist()])
    
    # Count pos cash accounts
    pos_agg['POS_COUNT'] = pos.groupby('SK_ID_CURR').size()
    
    del pos
    gc.collect()
    
    return pos_agg

<div style="background-color: #6D83C5;" >
    <h3 style="margin: auto; padding: 20px; color:#fff; ">Installments payments</h3>
</div>

<div class="alert alert-block alert-info">
    <p>History of payments made by the customer</p>
    <p>Each row/sample corresponds to a payment made or late payment</p>
</div>

In [8]:
# Preprocess installments_payments.csv
def installments_payments(num_rows=None, nan_as_category=True):
    
    # Read data
    ins = pd.read_csv(r"datasets\initials_datasets\installments_payments.csv", nrows=num_rows)
    
    # Categorical features with One-Hot encode
    ins, cat_cols = one_hot_encoder(ins, nan_as_category=True)
    
    # Percentage and difference paid in each installment (amount paid and installment value)
    ins['PAYMENT_PERC'] = ins['AMT_PAYMENT'] / ins['AMT_INSTALMENT']
    ins['PAYMENT_DIFF'] = ins['AMT_INSTALMENT'] - ins['AMT_PAYMENT']
    
    # Days past due and days before due (no negative values)
    ins['DPD'] = ins['DAYS_ENTRY_PAYMENT'] - ins['DAYS_INSTALMENT']
    ins['DBD'] = ins['DAYS_INSTALMENT'] - ins['DAYS_ENTRY_PAYMENT']
    ins['DPD'] = ins['DPD'].apply(lambda x: x if x > 0 else 0)
    ins['DBD'] = ins['DBD'].apply(lambda x: x if x > 0 else 0)
    
    # Features: Perform aggregations
    aggregations = {
        'NUM_INSTALMENT_VERSION': ['nunique'],
        'DPD': ['max', 'mean', 'sum'],
        'DBD': ['max', 'mean', 'sum'],
        'PAYMENT_PERC': ['max', 'mean', 'sum', 'var'],
        'PAYMENT_DIFF': ['max', 'mean', 'sum', 'var'],
        'AMT_INSTALMENT': ['max', 'mean', 'sum'],
        'AMT_PAYMENT': ['min', 'max', 'mean', 'sum'],
        'DAYS_ENTRY_PAYMENT': ['max', 'mean', 'sum']
    }
    
    for cat in cat_cols:
        aggregations[cat] = ['mean']
        
    ins_agg = ins.groupby('SK_ID_CURR').agg(aggregations)
    ins_agg.columns = pd.Index(['INSTAL_' + e[0] + "_" + e[1].upper() for e in ins_agg.columns.tolist()])
    
    # Count installments accounts
    ins_agg['INSTAL_COUNT'] = ins.groupby('SK_ID_CURR').size()
    
    del ins
    gc.collect()
    
    return ins_agg

<div style="background-color: #6D83C5;" >
    <h3 style="margin: auto; padding: 20px; color:#fff; ">Credit Card Balance</h3>
</div>

<div class="alert alert-block alert-info">
    <p>Monthly credit card balances</p>
</div>

In [9]:
# Preprocess credit_card_balance.csv
def credit_card_balance(num_rows=None, nan_as_category=True):
    
    # Read data
    cc = pd.read_csv(r"datasets\initials_datasets\credit_card_balance.csv", nrows=num_rows)
    
    # Categorical features with One-Hot encode
    cc, cat_cols = one_hot_encoder(cc, nan_as_category=True)
    
    # General aggregations
    cc.drop(['SK_ID_PREV'], axis= 1, inplace = True)
    cc_agg = cc.groupby('SK_ID_CURR').agg(['min', 'max', 'mean', 'sum', 'var'])
    cc_agg.columns = pd.Index(['CC_' + e[0] + "_" + e[1].upper() for e in cc_agg.columns.tolist()])
    
    # Count credit card lines
    cc_agg['CC_COUNT'] = cc.groupby('SK_ID_CURR').size()
    
    del cc
    gc.collect()
    
    return cc_agg

<div style="background-color: #506AB9;" >
    <h2 style="margin: auto; padding: 20px; color:#fff; ">3. Main function</h2>
</div>

<div class="alert alert-block alert-info">
    <p>Functions caller</p>
</div>

In [10]:
def main(debug=False):  
    
    # To test with less data
    num_rows = 10000 if debug else None
    
    with timer("Process Train/Test df"):    
        df = application_train_test(num_rows)
        print("\n")
        print(">> application_train_test")
        print("Train/Test df shape:", df.shape)
        #display(df.head())
    
    with timer("Process bureau and bureau_balance"):
        bureau = bureau_and_balance(num_rows)
        print("\n")
        print(">> bureau_and_balance")
        print("Bureau df shape:", bureau.shape)
        df = df.join(bureau, how='left', on='SK_ID_CURR')
        del bureau
        gc.collect()
        #display(df.head())
        
        
    with timer("Process previous_applications"):
        prev = previous_applications(num_rows)
        print("\n")
        print(">> previous_applications")
        print("Previous applications df shape:", prev.shape)
        df = df.join(prev, how='left', on='SK_ID_CURR')
        del prev
        gc.collect()
        #display(df.head())

        
    with timer("Process POS-CASH balance"):
        pos = pos_cash(num_rows)
        print("\n")
        print(">> pos_cash")
        print("Pos-cash balance df shape:", pos.shape)
        df = df.join(pos, how='left', on='SK_ID_CURR')
        del pos
        gc.collect()
        #display(df.head())
        

    with timer("Process installments payments"):
        ins = installments_payments(num_rows)
        print("\n")
        print(">> installments_payments")
        print("Installments payments df shape:", ins.shape)
        df = df.join(ins, how='left', on='SK_ID_CURR')
        del ins
        gc.collect()
        #display(df.head())
        
    with timer("Process credit card balance"):
        cc = credit_card_balance(num_rows)
        print("\n")
        print(">> credit_card_balance")
        print("Credit card balance df shape:", cc.shape)
        df = df.join(cc, how='left', on='SK_ID_CURR')
        del cc
        gc.collect()
        #display(df.head())
    
    # Calculating the size of the dataset on memory
    print("\n")
    result = memory_usage(df)
    print(">> Dataset on memory")
    print("  ", result)    
        
    return (df)


<div style="background-color: #506AB9;" >
    <h2 style="margin: auto; padding: 20px; color:#fff; ">5. Preprocessing's results</h2>
</div>

In [11]:
with timer("\n\nFull model run"):
    df = main()

Train samples: 307511, test samples: 48744


>> application_train_test
Train/Test df shape: (356251, 248)
Process Train/Test df - done in 5s


>> bureau_and_balance
Bureau df shape: (305811, 116)
Process bureau and bureau_balance - done in 22s


>> previous_applications
Previous applications df shape: (338857, 249)
Process previous_applications - done in 29s


>> pos_cash
Pos-cash balance df shape: (337252, 18)
Process POS-CASH balance - done in 17s


>> installments_payments
Installments payments df shape: (339587, 26)
Process installments payments - done in 38s


>> credit_card_balance
Credit card balance df shape: (103558, 141)
Process credit card balance - done in 28s


>> Dataset on memory
   memory usage: 1.8 GB


Full model run - done in 140s


<div style="background-color: #506AB9;" >
    <h2 style="margin: auto; padding: 20px; color:#fff; ">6 Optimizing memory usage</h2>
</div>

<div class="alert alert-block alert-warning">
    <p>We should optimize the memory usage to avoid problems during executions</p>
</div>

In [12]:
df_analysis(df, "df", analysis_type="header")


Analysis Header of df dataset
--------------------------------------------------------------------------------
- Dataset shape:			 356251 rows and 798 columns
- Total of NaN values:			 72099981
- Percentage of NaN:			 25.36 %
- Total of infinite values:		 21
- Percentage of infinite values:	 0.0 %
- Total of full duplicates rows:	 0
- Total of empty rows:			 0
- Total of empty columns:		 0
- Unique indexes:			 True
- Memory usage:				 1.8 GB


In [13]:
df["TARGET"].fillna(value=-99, inplace=True)
df["TARGET"] = df["TARGET"].astype("int8")
df["TARGET"] = df["TARGET"].replace(-99, np.nan)

In [14]:
df = memory_optimization(df)

In [15]:
df_analysis(df, "df", analysis_type="header")


Analysis Header of df dataset
--------------------------------------------------------------------------------
- Dataset shape:			 356251 rows and 798 columns
- Total of NaN values:			 72099981
- Percentage of NaN:			 25.36 %
- Total of infinite values:		 21
- Percentage of infinite values:	 0.0 %
- Total of full duplicates rows:	 0
- Total of empty rows:			 0
- Total of empty columns:		 0
- Unique indexes:			 True
- Memory usage:				 955.3 MB


<div class="alert alert-block alert-success">
    <p>We can see how the memory used has been reduced</p>
</div>

<div style="background-color: #506AB9;" >
    <h2 style="margin: auto; padding: 20px; color:#fff; ">7. Missing-values</h2>
</div>

<div class="alert alert-block alert-info">
    <p>Now, we are going to treat the missing-values in all the dataset, to do that, we are going to do the following steps</p>
    <ol>
        <li><b>Delete column with 20% or plus of Missing values</b></li>
        <li><b>Replace of infinite-values with NaN</b></li>
        <li><b>Fill missing-values with SimpleImputer</b> - There are 25 infinite values</li>
    </ol> 
    <p>Let's start by identifying the features with infinite-values and replace them by missing-values
   </p>
</div>
</div>

In [16]:
inf_cols = df.columns.to_series()[np.isinf(df).any()]

In [17]:
for col in inf_cols:
    df[col] = df[col].replace([np.inf, -np.inf], np.nan)

In [18]:
df_analysis(df, "df", analysis_type="header")


Analysis Header of df dataset
--------------------------------------------------------------------------------
- Dataset shape:			 356251 rows and 798 columns
- Total of NaN values:			 72100002
- Percentage of NaN:			 25.36 %
- Total of infinite values:		 0
- Percentage of infinite values:	 0.0 %
- Total of full duplicates rows:	 0
- Total of empty rows:			 0
- Total of empty columns:		 0
- Unique indexes:			 True
- Memory usage:				 955.3 MB


<div class="alert alert-block alert-info">
    <p>Now, we are going to delete all columns with 20% or plus of missing-values. But first, we have to save the TARGET variable
   </p>
</div>
</div>

In [19]:
df_temp = df["TARGET"].copy()

In [20]:
df = df.dropna(thresh=0.8*len(df), axis=1)

In [21]:
# Checking if TARGAT variable is still existing in the dataset
for col in df.columns:
    if col == "TARGET":
        print("Target variable exist")

Target variable exist


In [22]:
# Deleting the dataset temporal
del df_temp
gc.collect()

0

In [23]:
df_analysis(df, "df", analysis_type="header")


Analysis Header of df dataset
--------------------------------------------------------------------------------
- Dataset shape:			 356251 rows and 509 columns
- Total of NaN values:			 7678965
- Percentage of NaN:			 4.23 %
- Total of infinite values:		 0
- Percentage of infinite values:	 0.0 %
- Total of full duplicates rows:	 0
- Total of empty rows:			 0
- Total of empty columns:		 0
- Unique indexes:			 True
- Memory usage:				 546.2 MB


<div class="alert alert-block alert-success">
    <p>We have deleted 289 columns in the dataset
   </p>
</div>
</div>

<div class="alert alert-block alert-info">
    <p>Let's continue by identifying the features with missing-values and excluding the TARGET
   </p>
</div>
</div>

In [24]:
nan_cols = [i for i in df.columns if i!="TARGET" and df[i].isnull().any()]

In [25]:
for col in nan_cols:
    mean_value = df[col].mean()
    df[col].fillna(value=mean_value, inplace=True)

In [26]:
df_analysis(df, "df", analysis_type="header")


Analysis Header of df dataset
--------------------------------------------------------------------------------
- Dataset shape:			 356251 rows and 509 columns
- Total of NaN values:			 48744
- Percentage of NaN:			 0.03 %
- Total of infinite values:		 0
- Percentage of infinite values:	 0.0 %
- Total of full duplicates rows:	 0
- Total of empty rows:			 0
- Total of empty columns:		 0
- Unique indexes:			 True
- Memory usage:				 546.2 MB


<div class="alert alert-block alert-success">
    <p>Now, we only have missing values on the Target
   </p>
</div>
</div>

<div style="background-color: #506AB9;" >
    <h2 style="margin: auto; padding: 20px; color:#fff; ">8. Saving datasets</h2>
</div>

<div class="alert alert-block alert-info">
    <p>At this point, we are going to save current clients separate from clients to predict.</p>
    <p>But, before that, we are going to format columns name in the dataset</p>
</div>

In [27]:
# Formatting columns name
df = df.rename(columns=lambda x:re.sub("[^A-Za-z0-9_]+", "", x))

<div style="background-color: #506AB9;" >
    <h3 style="margin: auto; padding: 20px; color:#fff; ">8.1 Current clients</h3>
</div>

<div class="alert alert-block alert-info">
    <p>Now, let's save the current clients</p>
</div>
</div>

In [28]:
df_current_clients = df[df["TARGET"].notnull()]

In [29]:
df_analysis(df_current_clients, "df_current_clients", analysis_type="header")


Analysis Header of df_current_clients dataset
--------------------------------------------------------------------------------
- Dataset shape:			 307507 rows and 509 columns
- Total of NaN values:			 0
- Percentage of NaN:			 0.0 %
- Total of infinite values:		 0
- Percentage of infinite values:	 0.0 %
- Total of full duplicates rows:	 0
- Total of empty rows:			 0
- Total of empty columns:		 0
- Unique indexes:			 True
- Memory usage:				 464.5 MB


In [30]:
# saving the current clients
df_current_clients.to_csv("datasets/finals_datasets/df_current_clients.csv", index=False)

In [31]:
del df_current_clients
gc.collect()

0

<div style="background-color: #506AB9;" >
    <h3 style="margin: auto; padding: 20px; color:#fff; ">8.2 Clients to predict</h3>
</div>

<div class="alert alert-block alert-info">
    <p>Let's start saving the customers that we are going to predict</p>
</div>
</div>

In [32]:
df_clients_to_predict = df[df["TARGET"].isnull()]

In [33]:
df_analysis(df_clients_to_predict, "df_clients_to_predict", analysis_type="header")


Analysis Header of df_clients_to_predict dataset
--------------------------------------------------------------------------------
- Dataset shape:			 48744 rows and 509 columns
- Total of NaN values:			 48744
- Percentage of NaN:			 0.2 %
- Total of infinite values:		 0
- Percentage of infinite values:	 0.0 %
- Total of full duplicates rows:	 0
- Total of empty rows:			 0
- Total of empty columns:		 1
	+ The empty column is:		 ['TARGET']
- Unique indexes:			 True
- Memory usage:				 73.6 MB


In [34]:
# dropping TARGET feature
df_clients_to_predict = df_clients_to_predict.drop(columns=["TARGET"])

In [35]:
df_analysis(df_clients_to_predict, "df_clients_to_predict", analysis_type="header")


Analysis Header of df_clients_to_predict dataset
--------------------------------------------------------------------------------
- Dataset shape:			 48744 rows and 508 columns
- Total of NaN values:			 0
- Percentage of NaN:			 0.0 %
- Total of infinite values:		 0
- Percentage of infinite values:	 0.0 %
- Total of full duplicates rows:	 0
- Total of empty rows:			 0
- Total of empty columns:		 0
- Unique indexes:			 True
- Memory usage:				 74.5 MB


In [36]:
# saving the clients to predict
df_clients_to_predict.to_csv("datasets/finals_datasets/df_clients_to_predict.csv", index=False)

In [37]:
del df_clients_to_predict, df
gc.collect()

0